Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [237]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
import heapq
from icecream import ic

In [238]:
PUZZLE_DIM =3
action = namedtuple('Action', ['pos1', 'pos2'])

In [239]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

def is_solvable(state:np.ndarray)->bool:
    inv_count = 0
    tmp_state = state.copy().reshape(PUZZLE_DIM**2)
    for i in range(0,PUZZLE_DIM**2):
        for j in range(i+1,PUZZLE_DIM**2):
            if tmp_state[j] != 0 and tmp_state[i] != 0 and tmp_state[i] > tmp_state[j]:
                inv_count += 1
    return (inv_count%2) == 0 

In [240]:
RANDOMIZE_STEPS = 100_000
target_state=np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
start_state = target_state
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    start_state = do_action(start_state, choice(available_actions(start_state)))

start_state = np.array([np.array([0,2,1]),np.array([3,7,5]),np.array([8,6,4])])

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

In [241]:
class Position: #The position class represent the state of the game with the f (h+g) embedded.
    def __init__(self,grid:np.array,g,prev_pos=None) -> None:
        self.grid=grid
        self.prev_pos=prev_pos
        self.g=g
        self.f=int(self.__h()+g)
    def __h(self):
        #This is the heuristic: MANHATTAN DISTANCE over every element
        tot_dist=0
        for i in range(1,PUZZLE_DIM**2):
            x_1,y_1=np.where(target_state==i)
            x_2,y_2=np.where(self.grid==i)
            tot_dist+=abs(x_1-x_2)+abs(y_1-y_2)
        return tot_dist
    
    # #Overridng to use comparisons
    # def __eq__(self,other):
    #    return self.grid==other.grid
    
    #Overriding to use heap
    def __lt__(self, other):
     return self.f<other.f
    
    def __hash__(self):
            return hash((self.grid.tobytes(), self.f))

    
    def __eq__(self, other):
      
        return np.array_equal(self.grid, other.grid) and self.f==other.f






In [242]:
evaluated_pos=0#evaluated position (cost of the algorithm)

def a_star(start_pos:Position):
    global evaluated_pos
    evaluated_pos=0
    curr_pos=start_pos
    closed_list = set() #Visited Positions
    open_list=[] #Visitable Positions

    while open_list or evaluated_pos==0 :
        closed_list.add(curr_pos) #hashable
        if((curr_pos.grid==target_state).all()):
            ic(len(closed_list))
            ic(len(open_list))
            return curr_pos
        evaluated_pos+=1
        new_actions=available_actions(curr_pos.grid)
        for action in new_actions:
            new_pos=Position(do_action(curr_pos.grid,action),curr_pos.g+1,curr_pos)
            if new_pos not in closed_list:
                heapq.heappush(open_list, new_pos)
  
        curr_pos=heapq.heappop(open_list)
        # if(evaluated_pos%1000==0):
        #     ic(curr_pos.grid)

start_pos=Position(start_state,0)
ic(is_solvable(start_state))
ic("Start pos:")
ic(start_pos.grid,start_pos.f)

found_sol=a_star(start_pos)   


ic("Positions evaluated:",evaluated_pos)



ic("Steps:")
while True:
    ic(found_sol.grid)
    if(found_sol.prev_pos is None):
        break
    found_sol=found_sol.prev_pos

C:\Users\Ferra\AppData\Local\Temp\ipykernel_34320\515646168.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.f=int(self.__h()+g)
ic| is_solvable(start_state): True
ic| 'Start pos:'
ic| start_pos.grid: array([[0, 2, 1],
                           [3, 7, 5],
                           [8, 6, 4]])
    start_pos.f: 14
ic| len(closed_list): 1013
ic| len(open_list): 1753
ic| 'Positions evaluated:', evaluated_pos: 1687
ic| 'Steps:'
ic| found_sol.grid: array([[1, 2, 3],
                           [4, 5, 6],
                           [7, 8, 0]])
ic| found_sol.grid: array([[1, 2, 3],
                           [4, 5, 0],
                           [7, 8, 6]])
ic| found_sol.grid: array([[1, 2, 3],
                           [4, 0, 5],
                           [7, 8, 6]])
ic| found_sol.grid: array([[1, 2, 3],
   